In [17]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [18]:
def setDocumentContent():
    with open('/Users/sameernawaz/AnacondaProjects/Datasets/Streams.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        count = 0
        for line in csv_reader:
            val[count] = line
            count = count + 1
            if count >= sizeOfSets :
                break
            
    countTemp = 0
    for i in val:
        document[countTemp] = i[13]
        countTemp += 1

In [19]:
sizeOfSets = 300
val = [None] * sizeOfSets
document = [""] * sizeOfSets

setDocumentContent()

In [20]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
print(val[0])

['gFfpVo2FwT', '0', '', '', '', '1', '0', 'Pointer', '_User', 'JdOjsewt0L', '', '', '', 'Cheers ladies! Pleasure to meet you, I’m Safina, Society Gal’s Event Planner 🖤 Thrilled to connect with all of you here and always looking for feedback on what you want and how you want it! If I can find a way to bring it to you, be sure it’ll come in a sexy #safisticated package 💋', '7', '12', '0', '2020-03-03T12:49:31.508Z', '2020-03-03T12:49:31.508Z']


In [22]:
df = pd.DataFrame(val, columns = ['objectID', 'Shares', 'VideoType', 'VideoName', 'videoURL', 'Comments', 'profileClicks', 'userPointerType', 'userPointerClassName', 'userPointerObjectID', 'imageType', 'imageName', 'imageURL', 'text', 'likes', 'views', 'bookmarks', 'createdAt', 'updatedAt']) 

In [24]:
df.head()

,objectID,Shares,VideoType,VideoName,videoURL,Comments,profileClicks,userPointerType,userPointerClassName,userPointerObjectID,imageType,imageName,imageURL,text,likes,views,bookmarks,createdAt,updatedAt
0,gFfpVo2FwT,0,,,,1,0,Pointer,_User,JdOjsewt0L,,,,"Cheers ladies! Pleasure to meet you, I’m Safin...",7,12,0,2020-03-03T12:49:31.508Z,2020-03-03T12:49:31.508Z
1,jRNVujUa03,0,,,,0,1,Pointer,_User,XJ1fbtm3aC,File,652013e6864a6059b449f6b081709304_image.jpg,https://parsefiles.back4app.com/Eu2H7JiGkiBwPa...,Transform your life! Tomorrow Saturday Novembe...,6,7,1,2020-03-03T12:49:31.508Z,2020-03-03T12:49:31.508Z
2,uvvYbZtcsl,0,,,,1,0,Pointer,_User,MPIYycMk29,File,945e66a01ae64098f2b94937a134a566_image.jpg,https://parsefiles.back4app.com/Eu2H7JiGkiBwPa...,Making moves at the Digital Hollywood conventi...,6,22,0,2020-03-03T12:49:31.517Z,2020-03-03T12:49:31.517Z
3,h7GgZoqjRL,0,,,,0,1,Pointer,_User,EwlsOjH8zR,File,791db236ef99b9603c5dd2e6b5265292_image.jpg,https://parsefiles.back4app.com/Eu2H7JiGkiBwPa...,Note to self: All your milestones started from...,7,16,0,2020-03-03T12:49:31.517Z,2020-03-03T12:49:31.517Z
4,uFsYWSv0uZ,0,,,,1,1,Pointer,_User,DkROZ0bjmt,,,,Hello! So excited to be part of this! I’m Alis...,6,9,0,2020-03-03T12:49:31.517Z,2020-03-03T12:49:31.517Z


In [25]:
df = df[['objectID', 'userPointerObjectID', 'text']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 3 columns):
objectID               300 non-null object
userPointerObjectID    300 non-null object
text                   300 non-null object
dtypes: object(3)
memory usage: 9.4+ KB


In [26]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

300

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(300, 1953)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.000000,0.031253,0.000000,0.031348,0.000000,0.000000,0.000000,0.0,0.018056,0.00000,...,0.000000,0.074462,0.0,0.044366,0.0,0.000000,0.000000,0.000000,0.009750,0.038225
1,0.031253,1.000000,0.000000,0.000000,0.000000,0.066673,0.036348,0.0,0.036857,0.02975,...,0.000000,0.000000,0.0,0.000000,0.0,0.067165,0.000000,0.093421,0.019893,0.033656
2,0.000000,0.000000,1.000000,0.000000,0.088863,0.000000,0.000000,0.0,0.000000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.0,0.080018,0.000000,0.085184,0.026671,0.039878
3,0.031348,0.000000,0.000000,1.000000,0.019705,0.000000,0.000000,0.0,0.010130,0.00000,...,0.000000,0.063811,0.0,0.000000,0.0,0.000000,0.000000,0.019158,0.000000,0.098137
4,0.000000,0.000000,0.088863,0.019705,1.000000,0.020010,0.047376,0.0,0.029298,0.03607,...,0.091042,0.000000,0.0,0.000000,0.0,0.017008,0.018261,0.015002,0.037885,0.017022


In [29]:
movies_list = df['objectID'].values
movies_list, movies_list.shape

(array(['gFfpVo2FwT', 'jRNVujUa03', 'uvvYbZtcsl', 'h7GgZoqjRL',
        'uFsYWSv0uZ', 'F4gcyaeB0d', '4nBTSiQwHL', 'FPTz6YhQMb',
        'NvSHNuCrQh', 'EBCE9AXTQD', 's8Ckilgwk2', 'JlBd0kqVou',
        'ssuCcmPnTY', 'lJFFQiUN5l', 'Rf2iiUPE1H', 'LdIk9WxcGC',
        'HfBD0OFTBl', 'KIM2uTbCOz', '79sAJnCNfw', 'QtKac7HzJR',
        'zWzbMtOnfc', 's2p3C7g7SE', 'O1weceVmRs', '8BZVQFVwaT',
        'Ve6EkVJUG8', 'EhDhX22EvM', 'o0ZMcH9HaD', 'EkCSbT3w2x',
        'eJCsyOHrKM', 'VKKCXWAvkp', '7LwwopHrzt', 'NgslXBqwnK',
        'eSXB7yUFzm', 'Lg8DpLrxZL', 'EutKhxv4oF', 'edk4FU0i60',
        'ApBn2koyDG', 'xZ3TVkD04n', 'pAPBBch4rd', 'bQtJMlHovr',
        'MXZNm8JOxi', 'tgzfL8z0lr', 'Q1ReuDcEFT', 'QuIBtMT7fU',
        'TuZmkTGwQL', 'a0hpG87wrb', 'ftgGkuv6Nm', 'jyCdQiz9mA',
        'lAOjGGbAq7', 'bZxWPVTxTZ', 'olrNbG3dGR', 'x6MzHiCdFc',
        'eRgsUaGfF2', '9MDZgDmEiA', 'kltNeVnoCb', 'hOPYqKF7dA',
        'D2KcgpGjXf', 'EiDj2fqnqe', 'EoLb6GHQfp', 'jn3iLya8Kv',
        'CFjjpnRr4t', 'yxc6f6sih3', 'sKb

In [30]:
movie_idx = np.where(movies_list == 'hgFGyjWeqL')[0][0]
movie_idx

293

In [31]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0.04436607, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.11105727, 0.        , 0.        , 0.        ,
       0.05346409, 0.08911444, 0.04721582, 0.        , 0.03699367,
       0.        , 0.        , 0.08742843, 0.        , 0.04137212,
       0.        , 0.09743492, 0.02813608, 0.04342724, 0.09731969,
       0.        , 0.        , 0.03438401, 0.        , 0.        ,
       0.        , 0.06868949, 0.03810832, 0.08055643, 0.07645631,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06937478, 0.0862038 ,
       0.        , 0.09765571, 0.0648585 , 0.04947483, 0.07703283,
       0.02494847, 0.        , 0.05885204, 0.0568961 , 0.03795653,
       0.        , 0.        , 0.15244301, 0.        , 0.07627417,
       0.01665454, 0.02728139, 0.03967163, 0.2081278 , 0.        ,
       0.14054172, 0.07477449, 0.13332353, 0.        , 0.        ,
       0.03502361, 0.04310953, 0.07488064, 0.        , 0.     

In [32]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([176, 242,  63,  85, 192])

In [33]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['12raWayhMP', 'hjV9DZapBC', 'tMDC5qloAO', 'eS12AdiXmT',
       '71nlOrTVRZ'], dtype=object)

In [35]:
print((df.loc[df['objectID'] == "hgFGyjWeqL"]['text']).values[0])

Hey there 👋🏼💕 This post is for all my ladies! Focus on your goals. Get that money. Respect yourself & humble stunt on EVERYONE. XO’s Happy Hump Day 🙌🏼


In [37]:
print((df.loc[df['objectID'] == "hjV9DZapBC"]['text']).values[0])

If you aren’t selling something than you are not in business. Post your offers in the marketplace “buy & sell” and make that money. 
